# Map Dev for Reports

In [ ]:
import intake
import numpy as np
import pandas as pd
import geopandas as gpd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *
from shared_utils import geography_utils

import altair as alt

import _data_cleaning
import _report_utils

In [ ]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [ ]:
df = _report_utils.read_in_joined_data()

In [ ]:
pd.set_option("display.max_columns",500)

In [ ]:
mapsubset_cols= ['awarded','project_app_id', 'project_cycle', 'data_origin', 'geometry',
               'a1_imp_agcy_city','a1_imp_agcy_name','a1_proj_partner_agcy', 
               'assembly_district','congressional_district','senate_district',
              'a2_county', 'a2_ct_dist','a2_info_proj_name','a3_proj_type', 'total_atp_$', 'a2_proj_lat','a2_proj_long']

In [ ]:
df_map = _report_utils.fix_geom_issues(df, mapsubset_cols)

In [ ]:
df_map.explore("data_origin", cmap="tab20b")

In [ ]:
## from amanda's notebook for Project prioritization: https://github.com/cal-itp/data-analyses/blob/main/project_prioritization/_utils.py
shapes = "https://gis.data.ca.gov/datasets/0144574f750f4ccc88749004aca6eb0c_0.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
district_shapes = gpd.read_file(f"{shapes}").to_crs(epsg=4326)   

In [ ]:
#district_shapes.boundary.explore()

In [ ]:
## county geography

shapes2 = "https://opendata.arcgis.com/datasets/8713ced9b78a4abb97dc130a691a8695_0.geojson"
county_shapes = gpd.read_file(f"{shapes2}").to_crs(epsg=4326)

In [ ]:
#county_shapes.boundary.explore()

In [ ]:
df_map = df_map.drop(columns='index_right')

## Joining District and Points

In [ ]:
joined = district_shapes.merge(
    df_map, how="inner", left_on="DISTRICT", right_on="a2_ct_dist")

In [ ]:
joined.sample()

In [ ]:
sum_app = (joined>>group_by(_.DISTRICT, _.awarded)
 >>summarize(n_applications = _.project_app_id.nunique()))

In [ ]:
sum_app = sum_app.merge(
    district_shapes, how="inner", on="DISTRICT")

## Layering

In [ ]:
#base = district_shapes.plot(color='white', edgecolor='black')

#df_map.plot(ax=base, marker='o', color='blue', markersize=5)